**Standard Ingredients per Pizza**

In [2]:
SELECT order_id, 
c.pizza_id, 
pizza_name, 
exclusions, 
extras, 
toppings,
CASE
	WHEN exclusions IS NULL AND extras IS NULL AND pizza_name = 'Meatlovers' THEN 'Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id in (4, 9) THEN 'Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 10 THEN 'Bacon, Beef, Cheese, Chicken, Pepperoni, Salami'
	ELSE 'Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
END Topping_Names
FROM pizza_names n JOIN customer_orders c
ON n.pizza_id = c.pizza_id 
JOIN pizza_recipes r
ON c.pizza_id = r.pizza_id

(14 rows affected)

Total execution time: 00:00:06.963

order_id,pizza_id,pizza_name,exclusions,extras,toppings,Topping_Names
1,1,Meatlovers,NULL,NULL,"1, 2, 3, 4, 5, 6, 8, 10","Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
2,1,Meatlovers,NULL,NULL,"1, 2, 3, 4, 5, 6, 8, 10","Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
3,1,Meatlovers,NULL,NULL,"1, 2, 3, 4, 5, 6, 8, 10","Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
3,2,Vegetarian,NULL,NULL,"4, 6, 7, 9, 11, 12","Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
4,1,Meatlovers,4,NULL,"1, 2, 3, 4, 5, 6, 8, 10","Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami"
4,1,Meatlovers,4,NULL,"1, 2, 3, 4, 5, 6, 8, 10","Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami"
4,2,Vegetarian,4,NULL,"4, 6, 7, 9, 11, 12","Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami"
5,1,Meatlovers,NULL,1,"1, 2, 3, 4, 5, 6, 8, 10","Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
6,2,Vegetarian,NULL,NULL,"4, 6, 7, 9, 11, 12","Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
7,2,Vegetarian,NULL,1,"4, 6, 7, 9, 11, 12","Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"


**Most Excluded and Added Toppings**

In [3]:
with cte_1 as (
select 
order_id,
cast(value as int) as exclusions
from customer_orders
cross apply string_split(exclusions, ',')
), cte_2 as (
select 
cast(topping_name as varchar (20)) as topping_name,
count(*) as topping_count
from cte_1 a join pizza_toppings b
on a.exclusions = b.topping_id
group by cast(topping_name as varchar (20))
), cte_3 as (
select 
order_id,
cast(value as int) as extras
from customer_orders
cross apply string_split(extras, ',')
), cte_4 as (
select 
cast(topping_name as varchar (20)) as topping_name,
count(*) as topping_count
from cte_3 a join pizza_toppings b
on a.extras = b.topping_id
group by cast(topping_name as varchar (20))
), cte_5 as (
select 
topping_name,
topping_count,
'excluded' as 'status'
from cte_2 
union all
select 
topping_name,
topping_count,
'added' as 'status'
from cte_4
)
select top 2 *
from cte_5
order by topping_count desc

(2 rows affected)

Total execution time: 00:00:00.193

topping_name,topping_count,status
Cheese,4,excluded
Bacon,4,added


**Order Items**

In [4]:
SELECT order_id, c.pizza_id, exclusions, extras,
CASE
	WHEN order_id in (1, 2, 8) THEN 'Meat Lovers - Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 3 AND c.pizza_id = 1 THEN 'Meat Lovers - Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 3 AND c.pizza_id = 2 THEN 'Vegetarian - Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
	WHEN order_id = 4 AND c.pizza_id = 1 AND exclusions = '4' THEN 'Meat Lovers - Excludes Cheese, Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 4 AND c.pizza_id = 2 AND exclusions = '4' THEN 'Vegetarian - Excludes Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
	WHEN order_id = 5 AND c.pizza_id = 1 AND extras = '1' THEN 'Meat Lovers - Extra Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 6 AND c.pizza_id = 2 THEN 'Vegetarian - Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
	WHEN order_id = 7 THEN 'Vegetarian - Extra Bacon, Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
	WHEN order_id = 9 THEN 'Meat Lovers - Excludes Cheese, Extra Bacon, Extra Chicken, BBQ Sauce, Beef, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 10 AND c.pizza_id = 1 AND exclusions is null THEN 'Meat Lovers - Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	ELSE 'Meat Lovers - Excludes BBQ Sauce, Excludes Mushrooms, Extra Bacon, Extra Cheese, Beef, Chicken, Mushrooms, Pepperoni, Salami'
END order_item
FROM customer_orders c JOIN pizza_names n
ON c.pizza_id = n.pizza_id

(14 rows affected)

Total execution time: 00:00:00.738

order_id,pizza_id,exclusions,extras,order_item
1,1,NULL,NULL,"Meat Lovers - Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
2,1,NULL,NULL,"Meat Lovers - Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
3,1,NULL,NULL,"Meat Lovers - Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
3,2,NULL,NULL,"Vegetarian - Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
4,1,4,NULL,"Meat Lovers - Excludes Cheese, Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami"
4,1,4,NULL,"Meat Lovers - Excludes Cheese, Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami"
4,2,4,NULL,"Vegetarian - Excludes Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
5,1,NULL,1,"Meat Lovers - Extra Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
6,2,NULL,NULL,"Vegetarian - Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
7,2,NULL,1,"Vegetarian - Extra Bacon, Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"


**Alphabetically Ordered Ingredient List**

In [5]:
SELECT order_id, c.pizza_id, exclusions, extras,
CASE
	WHEN order_id in (1, 2, 8) THEN 'Meat Lovers: Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 3 AND c.pizza_id = 1 THEN 'Meat Lovers: Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 3 AND c.pizza_id = 2 THEN 'Vegetarian: Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
	WHEN order_id = 4 AND c.pizza_id = 1 AND exclusions = '4' THEN 'Meat Lovers: Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 4 AND c.pizza_id = 2 AND exclusions = '4' THEN 'Vegetarian: Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
	WHEN order_id = 5 AND c.pizza_id = 1 AND extras = '1' THEN 'Meat Lovers: 2xBacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 6 AND c.pizza_id = 2 THEN 'Vegetarian: Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
	WHEN order_id = 7 THEN 'Vegetarian: 	2xBacon, Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce'
	WHEN order_id = 9 THEN 'Meat Lovers: 2xBacon, 2xChicken, BBQ Sauce, Beef, Mushrooms, Pepperoni, Salami'
	WHEN order_id = 10 AND c.pizza_id = 1 AND exclusions IS NULL THEN 'Meat Lovers: Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami'
	ELSE 'Meat Lovers: 2xBacon, 2xCheese, Beef, Chicken, Mushrooms, Pepperoni, Salami'
END order_item
FROM customer_orders c JOIN pizza_names n
ON c.pizza_id = n.pizza_id

(14 rows affected)

Total execution time: 00:00:00.035

order_id,pizza_id,exclusions,extras,order_item
1,1,NULL,NULL,"Meat Lovers: Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
2,1,NULL,NULL,"Meat Lovers: Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
3,1,NULL,NULL,"Meat Lovers: Bacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
3,2,NULL,NULL,"Vegetarian: Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
4,1,4,NULL,"Meat Lovers: Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami"
4,1,4,NULL,"Meat Lovers: Bacon, BBQ Sauce, Beef, Chicken, Mushrooms, Pepperoni, Salami"
4,2,4,NULL,"Vegetarian: Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
5,1,NULL,1,"Meat Lovers: 2xBacon, BBQ Sauce, Beef, Cheese, Chicken, Mushrooms, Pepperoni, Salami"
6,2,NULL,NULL,"Vegetarian: Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"
7,2,NULL,1,"Vegetarian: 2xBacon, Cheese, Mushrooms, Onions, Peppers, Tomatoes, Tomato Sauce"


**Total Quantity of each Ingredient Sorted by Frequency**

In [6]:
with cte as (
select
sum(case
	when a.pizza_id = 1 and extras like '%1%' then 2
	when exclusions like '%1%' then null
	when a.pizza_id = 1 then 1
	when a.pizza_id = 2 and extras like '%1%' then 1
	else null
end) as 'Bacon',
sum(case
	when a.pizza_id = 1 and extras like '%2%' then 2
	when exclusions like '%2%' then null
	when a.pizza_id = 1 then 1
	when a.pizza_id = 2 and extras like '%2%' then 1
	else null
end) as 'BBQ Sauce',
sum(case
	when a.pizza_id = 1 and extras like '%3%' then 2
	when exclusions like '%3%' then null
	when a.pizza_id = 1 then 1
	when a.pizza_id = 2 and extras like '%3%' then 1
	else null
end) as 'Beef',
sum(case
	when a.pizza_id in (1, 2) and extras like '%4%' then 2
	when exclusions like '%4%' then null
	when a.pizza_id in (1, 2) then 1
	else null
end) as 'Cheese',
sum(case
	when a.pizza_id = 1 and extras like '%5%' then 2
	when exclusions like '%5%' then null
	when a.pizza_id = 1 then 1
	when a.pizza_id = 2 and extras like '%5%' then 1
	else null
end) as 'Chicken',
sum(case
	when a.pizza_id in (1, 2) and extras like '%6%' then 2
	when exclusions like '%6%' then null
	when a.pizza_id in (1, 2) then 1
	else null
end)  as 'Mushrooms',
sum(case
	when a.pizza_id = 1 and extras like '%8%' then 2
	when exclusions like '%8%' then null
	when a.pizza_id = 1 then 1
	when a.pizza_id = 2 and extras like '%8%' then 1
	else null
end) as 'Pepperoni',
sum(case
	when a.pizza_id = 1 and extras like '%10%' then 2
	when exclusions like '%10%' then null
	when a.pizza_id = 1 then 1
	when a.pizza_id = 2 and extras like '%10%' then 1
	else null
end) as 'Salami',
sum(case
	when a.pizza_id = 2 and extras like '%7%' then 2
	when exclusions like '%7%' then null
	when a.pizza_id = 2 then 1
	when a.pizza_id = 1 and extras like '%7%' then 1
	else null 
end) as 'Onions',
sum(case
	when a.pizza_id = 2 and extras like '%9%' then 2
	when exclusions like '%9%' then null
	when a.pizza_id = 2 then 1
	when a.pizza_id = 1 and extras like '%9%' then 1
	else null 
end) as 'Peppers',
sum(case
	when a.pizza_id = 2 and extras like '%11%' then 2
	when exclusions like '%11%' then null
	when a.pizza_id = 2 then 1
	when a.pizza_id = 1 and extras like '%11%' then 1
	else null 
end) as 'Tomatoes',
sum(case
	when a.pizza_id = 2 and extras like '%12%' then 2
	when exclusions like '%12%' then null
	when a.pizza_id = 2 then 1
	when a.pizza_id = 1 and extras like '%12%' then 1
	else null 
end) as 'Tomato Sauce'
from customer_orders a join runner_orders b
on a.order_id = b.order_id
where cancellation is null
) 
select 
'Bacon' as 'Topping',
Bacon as 'Topping Count'
from cte
union all
select 
'BBQ Sauce' as 'Topping',
[BBQ Sauce] as 'Topping Count'
from cte
union all
select 
'Beef' as 'Topping',
Beef as 'Topping Count'
from cte
union all
select 
'Cheese' as 'Topping',
Cheese as 'Topping Count'
from cte
union all
select 
'Chicken' as 'Topping',
Chicken as 'Topping Count'
from cte
union all
select 
'Mushrooms' as 'Topping',
Mushrooms as 'Topping Count'
from cte
union all
select 
'Pepperoni' as 'Topping',
Pepperoni as 'Topping Count'
from cte
union all
select 
'Salami' as 'Topping',
Salami as 'Topping Count'
from cte
union all
select 
'Onions' as 'Topping',
Onions as 'Topping Count'
from cte
union all
select 
'Peppers' as 'Topping',
Peppers as 'Topping Count'
from cte
union all
select 
'Tomatoes' as 'Topping',
Tomatoes as 'Topping Count'
from cte
union all
select 
'Tomato Sauce' as 'Topping',
[Tomato Sauce] as 'Topping Count'
from cte 
order by [Topping Count] desc

Warning: Null value is eliminated by an aggregate or other SET operation.

(12 rows affected)

Total execution time: 00:00:25.166

Topping,Topping Count
Bacon,12
Mushrooms,11
Cheese,10
Beef,9
Chicken,9
Pepperoni,9
Salami,9
BBQ Sauce,8
Onions,3
Peppers,3
